In [1]:
import numpy as np
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue


In [2]:
# constants
PROBLEM_SIZE = 5  # dimension of the finite set U
NUMBER_SET = 10  # number of subsets in the collection S
SETS = tuple(
    np.array([random() < 0.3 for i in range(PROBLEM_SIZE)]) for j in range(NUMBER_SET)
)  # generate sets in S

# Define State as a named tuple
State = namedtuple("State", ["taken", "cost", "heuristic"])


In [3]:
def goal_check(state, sets):
    """
    check if the logical OR all the elements yeald a line of all true ie the
    condition for a state to be covering the whole set U
    """
    return np.all(
        reduce(np.logical_or, [sets[i] for i in state.taken], np.zeros(PROBLEM_SIZE))
    )


# assert generated problem is solvable, ie the goal check of a stete with all
# sets taken is true
assert goal_check(State(range(NUMBER_SET), 0, 0), SETS)


In [4]:
def cost(state):
    """The cost function calculates the cost of reaching a particular state"""
    return len(state.taken)

Commonly used Heuristic for set covering:

- Minimum Remaining Elements (MRE): Prioritizes subsets with the fewest uncovered elements.

- Maximum Coverage (MC): Favors subsets that cover the maximum number of uncovered elements.

- Combination of MRE and MC: Balances between subsets with few remaining elements and high coverage.

- Randomized Heuristics: Selects subsets randomly or based on probabilistic considerations to explore different paths.


In [5]:
#
# state = namedtuple("State", ["taken", "cost", "heuristic"])
#
# SETS example =
#   Set 0 [False  True  True  True False] Coverage: 3
#   Set 1 [False  True False False False] Coverage: 1
#   Set 2 [ True  True False False False] Coverage: 2
#   Set 3 [ True False False False  True] Coverage: 2
#   Set 4 [False  True  True  True False] Coverage: 3
#   Set 5 [ True False False False False] Coverage: 1
#   Set 6 [False False False False False] Coverage: 0
#   Set 7 [False False False False  True] Coverage: 1
#   Set 8 [False False False False False] Coverage: 0
#   Set 9 [False  True False False False] Coverage: 1
#


def MRE_heuristic(state, sets):
    """
    - Objective: The MRE heuristic prioritizes selecting subsets that have the
      fewest remaining uncovered elements.
    - Selection Criterion: It selects subsets that cover elements that are closest
    to being fully covered. In other words, it aims to quickly cover the remaining
    elements.
    - Impact: The primary goal is to minimize the number of remaining uncovered
    elements
    """
    uncovered = reduce(
        np.logical_or, [sets[i] for i in state.taken], np.zeros(PROBLEM_SIZE)
    )

    heuristic = -np.sum(np.logical_not(uncovered))
    return heuristic


def MC_heuristic(state, sets):
    """
    - Objective: The MC heuristic prioritizes selecting subsets that cover the
    maximum number of currently uncovered elements.
    - Selection Criterion: It
    selects subsets that contribute to covering the largest number of currently
    uncovered elements.
    - Impact: The primary goal is to maximize the coverage
    with each selection
    """
    uncovered = reduce(
        np.logical_or, [sets[i] for i in state.taken], np.zeros(PROBLEM_SIZE)
    )

    heuristic = -np.sum(uncovered)
    return heuristic


def MRE_MC_heuristic(state, sets):
    """
    Prioritize with a balanced approach between MRE and MC
    """
    return (MRE_heuristic(state, sets) + MC_heuristic(state, sets)) / 2


def RANDOM_heuristic(state, sets):
    """
    Prioritize with a random heuristic
    """
    return random()


def COSTONLY_heuristic(state, sets):
    """
    Prioritize only with cost
    """
    return cost(state)


In [6]:
def astar(sets, heuristic):
    # Initialize the priority queue with the initial state
    initial_state = State(
        taken=[],
        cost=0,
        heuristic=heuristic(State(taken=[], cost=0, heuristic=0), sets),
    )
    open_set = PriorityQueue()
    open_set.put((initial_state.cost + initial_state.heuristic, initial_state))

    # Initialize the closed set as an empty set
    closed_set = set()

    checked_states = 0

    while not open_set.empty():
        # Get the state with the lowest f score from the priority queue
        _, current_state = open_set.get()

        checked_states += 1

        # If the current state is a goal state, return the solution
        if goal_check(current_state, sets):
            return current_state.taken, checked_states

        # Add the current state to the closed set
        closed_set.add(tuple(current_state.taken))

        # Generate successor states by adding one more subset
        for subset in range(NUMBER_SET):
            if subset not in current_state.taken:
                # Create a new state by adding the subset
                new_taken = current_state.taken + [subset]
                new_cost = cost(State(new_taken, 0, 0))
                new_heuristic = heuristic(State(new_taken, 0, 0), sets)
                new_state = State(new_taken, new_cost, new_heuristic)

                # If the state is not in the closed set, add it to the open set
                if tuple(new_taken) not in closed_set:
                    open_set.put((new_state.cost + new_state.heuristic, new_state))

    # If the open set is empty and no solution is found, return None
    return None


In [8]:
for i in range(NUMBER_SET):
    print("Set", i, SETS[i], "Coverage:", np.sum(SETS[i]))

print()
print("MRE:", astar(SETS, MRE_heuristic))
print("MC:", astar(SETS, MC_heuristic))
print("MRE+MC:", astar(SETS, MRE_MC_heuristic))
print("RANDOM:", astar(SETS, RANDOM_heuristic))
print(
    "COSTONLY:", astar(SETS, COSTONLY_heuristic)
)  # cost only seems to be consistently achieving the same result as MRE+MC


Set 0 [False False False  True  True] Coverage: 2
Set 1 [False  True False False False] Coverage: 1
Set 2 [False False  True False False] Coverage: 1
Set 3 [False False False False False] Coverage: 0
Set 4 [False  True False False False] Coverage: 1
Set 5 [False False  True  True False] Coverage: 2
Set 6 [False False False False False] Coverage: 0
Set 7 [False False False  True  True] Coverage: 2
Set 8 [False False False False False] Coverage: 0
Set 9 [ True  True False  True  True] Coverage: 4

MRE: ([2, 9], 2048)
MC: ([9, 2], 3)
MRE+MC: ([2, 9], 38)
RANDOM: ([2, 9], 16)
COSTONLY: ([2, 9], 38)


#### Comments

- MRE consistently takes a lot of step, seems to make sense has it gives more priority to sets who cover a lot of already covered spaces

- MC consistently behaves the best

- MRE + MC behaves a bit worse than MC but a lot better than MRE

- RANDOM seems to perform on par with MRE + MC

- COSTONLY behaves exactly like MRE + MC, maybe because both are trying to maximize the coverage of elements (by minimizing the number of uncovered elements) and the cost function itself already captures the essence of minimizing the number of selected subsets.


# AVG Steps


In [9]:
steps_MRE = []
steps_MC = []
steps_MRE_MC = []
steps_RANDOM = []
steps_COSTONLY = []

N_RUNS = 20

PROBLEM_SIZE = 5  # dimension of the finite set U
NUMBER_SET = 10  # number of subsets in the collection S

print(f"Problem size: {PROBLEM_SIZE} - Number of sets: {NUMBER_SET}")

for _ in range(N_RUNS):
    sets = tuple(
        np.array([random() < 0.3 for i in range(PROBLEM_SIZE)])
        for j in range(NUMBER_SET)
    )

    if goal_check(State(range(NUMBER_SET), 0, 0), sets):
        steps_MRE.append(astar(sets, MRE_heuristic)[1])
        steps_MC.append(astar(sets, MC_heuristic)[1])
        steps_MRE_MC.append(astar(sets, MRE_MC_heuristic)[1])
        steps_RANDOM.append(astar(sets, RANDOM_heuristic)[1])
        steps_COSTONLY.append(astar(sets, COSTONLY_heuristic)[1])
        print(f"Run {_+1} completed")
    else:
        print(f"Run {_+1} not solvable")


print(f"Average number of steps over {N_RUNS} runs:")
print(f"MRE: {sum(steps_MRE)/len(steps_MRE):.2f}")
print(f"MC: {sum(steps_MC)/len(steps_MC):.2f}")
print(f"MRE+MC: {sum(steps_MRE_MC)/len(steps_MRE_MC):.2f}")
print(f"RANDOM: {sum(steps_RANDOM)/len(steps_RANDOM):.2f}")
print(f"COSTONLY: {sum(steps_COSTONLY)/len(steps_COSTONLY):.2f}")


Problem size: 5 - Number of sets: 10
Run 1 completed
Run 2 completed
Run 3 completed
Run 4 completed
Run 5 completed
Run 6 completed
Run 7 completed
Run 8 not solvable
Run 9 completed
Run 10 completed
Run 11 not solvable
Run 12 completed
Run 13 completed
Run 14 completed
Run 15 completed
Run 16 completed
Run 17 completed
Run 18 not solvable
Run 19 completed
Run 20 completed
Average number of steps over 20 runs:
MRE: 2599.59
MC: 3.47
MRE+MC: 84.41
RANDOM: 79.06
COSTONLY: 84.41
